In [ ]:
import numpy as np
import pandas as pd
import joblib
from keras.models import load_model  
import os

MODEL_PATH  = 'model.h5'
SCALER_PATH = 'scaler.pkl'
DATA_PATH   = 'data.csv'

if not (os.path.exists(MODEL_PATH) and os.path.exists(SCALER_PATH) and os.path.exists(DATA_PATH)):
    raise FileNotFoundError("Make sure model.h5, scaler.pkl, and data.csv are in the working directory")


model  = load_model(MODEL_PATH)
scaler = joblib.load(SCALER_PATH)
df     = pd.read_csv(DATA_PATH)

feature_names = [c for c in df.columns 
                 if c not in ('id', 'diagnosis')]
print("=== Valid Input Ranges ===")
min_vals = df[feature_names].min()
max_vals = df[feature_names].max()
for feat in feature_names:
    print(f"{feat:20s}  Min = {min_vals[feat]:8.4f}  Max = {max_vals[feat]:8.4f}")

print("\nPlease enter your values (within the ranges above).")
user_vals = []
for feat in feature_names:
    low, high = min_vals[feat], max_vals[feat]
    while True:
        try:
            val = float(input(f"{feat} [{low:.4f}–{high:.4f}]: "))
        except ValueError:
            print(" Invalid number. Try again.")
            continue
        if val < low or val > high:
            print(f" Out of range! Must be between {low:.4f} and {high:.4f}.")
        else:
            print(f"You entered: {feat} = {val:.4f}")
            user_vals.append(val)
            break

user_array = np.array(user_vals)
print(f"\nYour input summary: min = {user_array.min():.4f}, max = {user_array.max():.4f}")
user_df     = pd.DataFrame([user_vals], columns=feature_names)
user_scaled = scaler.transform(user_df)
pred_prob   = model.predict(user_scaled)[0][0]
prediction  = "Malignant" if pred_prob > 0.5 else "Benign"
confidence  = pred_prob if pred_prob > 0.5 else (1 - pred_prob)

print("\n=== Prediction ===")
print(f"Predicted Cancer Type : {prediction}")
print(f"Confidence Score       : {confidence:.4f}")
